In [1]:
def GetTweets(SearchTerms, MaxNumberOfTweets):
    import pandas as pd
    import numpy as np
    from twython import Twython
    import pickle
    import urllib
    
    # Get keys for Twitter Developer API
    with open("TwitterKeys.dat", "rb") as SaveFile:
        ReadData = pickle.load(file=SaveFile)

    twitterStream = Twython(ReadData["ConsumerKey"],ReadData["ConsumerSecret"])
    
    # Loop through stream until dry or max number of tweets reached
    twitterData = []
    max_id = int(np.iinfo(np.int64).max)

    maxQueryCount = int(MaxNumberOfTweets / 100) * 2  # Limit of 100 tweets per request. Arbitrarily query twitter at least 2x as often
    readTweetCount = 0
    
    for loopCount in range(maxQueryCount):
        # Pull data from stream
        streamData = twitterStream.search(
            q=str(["\"" + term + "\" " for term in SearchTerms]) + "-filter:retweets AND -filter:replies", # Querystring, removing potential duplicates
            max_id = max_id, # Get next set of tweets
            tweet_mode = "extended", # 280 char support instead of legacy 140 char
            count=MaxNumberOfTweets,
        )
        print("Loading ", str(len(streamData["statuses"])), " tweets.")
        # Save tweets
        twitterData.append(streamData["statuses"])
        readTweetCount += len(streamData["statuses"])

        # Get max_id and update marker
        max_id = streamData["search_metadata"].get("next_results")
        if(max_id):
            max_id = urllib.parse.parse_qs(max_id.replace("?", "")).get("max_id")
            if(max_id):
                max_id = int(max_id[0])
            else:
                break
        else:
            break
        
        # Check if got enough tweets
        if(readTweetCount >= MaxNumberOfTweets):
            break

    # Return flattened list of tweets (statuses)
    twitterData = pd.DataFrame([tweet for tweetGroup in twitterData for tweet in tweetGroup])

    print("")
    print("Dupes: ", str(twitterData["id"].duplicated().sum()))
    print("")
    display(twitterData.info())
    display(twitterData.head())
    
    return twitterData


In [5]:
TwitterData = GetTweets(["food"], 10000)

# Save tweets to file
import pickle
with open("TwitterRawData-Food.dat", "wb") as SaveFile:
    pickle.dump(TwitterData, file=SaveFile)

Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  1

None

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status
0,Thu Jan 30 14:59:30 +0000 2020,1222897127476449283,1222897127476449283,From Discover on Google https://t.co/Z6X7dxfZYq,False,"[0, 47]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",None,...,0,0,False,False,False,en,NaN,NaN,NaN,NaN
1,Thu Jan 30 14:59:29 +0000 2020,1222897126625038339,1222897126625038339,Just getting to work at tandooritasteofindia f...,False,"[0, 234]","{'hashtags': [{'text': 'indianeats', 'indices'...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://instagram.com"" rel=""nofollow"">...",None,...,0,0,False,False,False,en,NaN,NaN,NaN,NaN
2,Thu Jan 30 14:59:29 +0000 2020,1222897126419521538,1222897126419521538,"""Golden Milk"" #food #eat #health #tastyfix htt...",False,"[0, 66]","{'hashtags': [{'text': 'food', 'indices': [14,...","{'iso_language_code': 'da', 'result_type': 're...","<a href=""http://engage.social"" rel=""nofollow"">...",None,...,0,0,False,False,False,da,NaN,NaN,NaN,NaN
3,Thu Jan 30 14:59:29 +0000 2020,1222897125962342401,1222897125962342401,Halfway to work this morning I looked over at ...,False,"[0, 273]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",None,...,0,0,False,False,NaN,en,NaN,NaN,NaN,NaN
4,Thu Jan 30 14:59:29 +0000 2020,1222897125144399878,1222897125144399878,lol guy can’t have any liquids after 7:30am. H...,False,"[0, 148]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",None,...,0,0,False,False,False,en,"{'media': [{'id': 1222897118802612224, 'id_str...",NaN,NaN,NaN
